<a href="https://colab.research.google.com/github/Suji04/NormalizedNerd/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##        Ce fichier va générer les données attendues par Spacy pour entrainement (modèle avec toutes les EN).







_N.B.: il s'agit ici du cas pour les ingrédients uniquement_

In [1]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import string

### Load Data

In [3]:
import pandas as pd

In [4]:
data= pd.read_json('./Data/jsonCorpusRecettes.json')

In [5]:
data.head()

,titre,preparation,ingredients
0,Poulet tandoori rouge,A préparer la veille !Mélanger l'épice Tandoor...,"[4 escalopes de poulet, 4 cuillères à soupe d'..."
1,Lasagnes végétariennes (facile),"Si vous utilisez des oignons, faites-les reven...","[6 tomates fraîches (ou pelées en boîte, à déf..."
2,Terrine de foie de porc,"- hacher (pas trop fin) le foie, le lard, les ...","[400 grammes de foie de porc , 200 grammes de ..."
3,Lapin au vin blanc (à la cocotte),Faire revenir les échalotes et les lardons dan...,"[1,8 kg de lapin (7 à 8 morceaux), 1 bouteille..."
4,Escalopes lucullus à la cocotte,"Coupez l'oignon en petits morceaux, faites rev...","[4 escalopes, 4 tranches de bacon, 4 tranches ..."


In [6]:
df = data.copy()
# Format du Dataframe
df.shape

(23071, 3)


- Chaque instruction (préparation) se compose de plusieurs phrases: on découpe en phrases.


In [7]:
import re, string

In [8]:
tmp=[]
for i in range(df.shape[0]):    
    tmp = tmp + re.split("[" +'!?.'+ "]+" , df.preparation[i])


In [9]:
tmpTitre=[]
for i in range(df.shape[0]):    
    tmpTitre = tmpTitre + re.split("[" +'!?.'+ "]+" , df.titre[i])

In [10]:
len(tmp),len(tmpTitre)

(247190, 23226)

=> On constate que toutes les recettes n'ont pas de titre.

On commence le prétraitement: en décollant et séparant les ponctuations dans un premier temps (pour la prépération mais aussi les titres) et en effaçant la casse (on passe tout les textes en minuscules).

In [11]:
listPhrases=[]
for i in range(len(tmp)):  
    if tmp[i] != '':
        listPhrases.append(tmp[i].replace('-',' - ').replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace('\'','  ').replace('\n','').replace('(',' ( ').replace(')',' ) ').replace('"',' ').lower())


In [12]:
listPhrasesTitres=[]
for i in range(len(tmpTitre)):  
    if tmpTitre[i] != '':
        listPhrasesTitres.append(tmpTitre[i].replace('-',' - ').replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace('\'','  ').replace('\n','').replace('(',' ( ').replace(')',' ) ').replace('"',' ').lower())


In [13]:
import json
import nltk
from nltk.stem.snowball import FrenchStemmer
import csv

In [50]:
# Méthode qui prend en entrée un fichier de stopwords, qui le charge et renvoie la liste des stopwords
def readJsonStopWordsInList(fileFrenchStopWords):
    dataDict={}
    with open(fileFrenchStopWords,encoding="utf8") as json_data:
        dataDict = json.load(json_data)
    return dataDict

In [51]:
# Normalisation, tokenisation, suppression des données non alphabetiques
def initialNormalizeAndTokenize(phrase_test):

    word_text =[]
    word_text = nltk.word_tokenize(phrase_test)

    # on peut enlever les ponctuations ce n est pas une analyse de sentiments.. 
    ponc ="!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»«…\ '’"
    table = str.maketrans("","", ponc)

    # convert to lower case
    stripped = [w.translate(table).lower() for w in word_text]   
        
    # remove remaining tokens that are not alphabetic
    #stripped = [word for word in stripped if word.isalpha()]
    
    assembled =" ".join(stripped)
    assembled = assembled.rstrip()
    word_text = nltk.word_tokenize(assembled)    
    return word_text

In [52]:
# Lemmatiseur (stemmer) en français
def initialLemmatizer(word_text):

    # lemmatizer = FrenchLefffLemmatizer()
    # lemWords = [lemmatizer.lemmatize(word) for word in word_text]
    stemmer = FrenchStemmer()
    lemWords = [stemmer.stem(word) for word in word_text]

    return lemWords 

In [53]:
# Méthode qui va prétraiter en utilisant le radical (stemmer) en particulier
def preprocessingUsingNorm(stopWords,text,adjWords):

    textNorm=[]
    for i,val in enumerate(text):
        phrase_test=val     

        # tokenization, suppression ponctuation et normalisation
        word_text = initialNormalizeAndTokenize(phrase_test)
    
        # stopWords supprimes et qq adjectifs et adverbes
        words = initialBow(stopWords,word_text,adjWords)
        
        
        # II) Lemmatisation (il s'agit de Stemmer ici mais c'est équivalent)
        lemWords = initialLemmatizer(words) 
        
        #stripped = unique(words)
        assembled =" ".join(lemWords)
        assembled = assembled.rstrip()
        textNorm.append(assembled)   
        
    return textNorm



In [54]:
# Méthode qui va supprimer les stopwords mais aussi une liste de mots (adjectifs, adverbes..)
def initialBow(stopWords,word_text,adjWords):
    tmp = [w for w in word_text if not w in stopWords]
    words = [w for w in tmp if not w in adjWords]
    return words

In [55]:
fileFrenchStopWords = "./Data/stopwords-fr.json"

In [56]:
# on charge les stopWords

stopWords={}
stopWords = readJsonStopWordsInList(fileFrenchStopWords) 


In [57]:
# liste de token à enlever de la liste également: ajectifs, adverbes...

df_Adj =  pd.read_csv('./Data/adjectifCulinaire.txt')

In [58]:
# On met en lowercase
for i in range(df_Adj.shape[0]):    
    df_Adj.Liste_A_Enlever[i] = df_Adj['Liste_A_Enlever'][i].lower()

In [59]:
df_Adj.head()

,Liste_A_Enlever
0,succulent
1,savoureux
2,aromatique
3,allégé
4,goûteux


In [60]:
adjWords =  df_Adj['Liste_A_Enlever'].values.tolist()

### On normalise , on enlève les stopwords, des ajectifs et adverbes, les nombres, on lemmatise (stemmer)

In [61]:
textNorm = preprocessingUsingNorm(stopWords,listPhrases,adjWords)  

In [62]:
len(textNorm)


230494

In [63]:
listPhrasesTitresNorm = preprocessingUsingNorm(stopWords,listPhrasesTitres,adjWords)  

In [64]:
# Récupération des données de sorties de la clusturisation: une liste d'ingrédients (radical)

df_ingredientsSave =  pd.read_csv('./listIngredientsToSave.csv')

In [65]:
df_ingredientsSave.head()

,IngredientsSave
0,limoncello
1,épinard
2,combav
3,rapadur
4,mikados


In [66]:
df_ingredientsSave.shape

(1127, 1)

In [67]:
# transformation en liste
ingredients = df_ingredientsSave['IngredientsSave'].values.tolist()


In [68]:
# récupération de la liste des ustensiles
ustensiles = []
with open("./Data/ustensiles.txt", "r",encoding="utf-8") as file:
    for item in file:
        item = item.split("\n")[0].lower()
        if item != '':
            ustensiles.append(item)

In [69]:
#stem
stemmer = FrenchStemmer()
ustensiles = [stemmer.stem(word) for word in ustensiles]
len(ustensiles)

210

In [70]:
#récupération de la liste des verbes
verbes = []
with open("./Data/verbesSpacyCorrecPlusNet.txt", "r",encoding="utf-8") as file:
    for item in file:
        item = item.split("\n")[0].lower()
        if item != '':
            verbes.append(item)

In [71]:
#stem
verbes = [stemmer.stem(word) for word in verbes]
len(verbes)
#verbes

440

## Spacy : EntityRuler et le label 'INGREDIENT' 

- Utilisation de _EntityRuler_ pour créer l'entité _'INGREDIENT'_ et faire le lien avec notre liste d'ingrédients.

In [36]:
import spacy
from spacy.language import Language
from spacy.pipeline import EntityRuler

In [37]:
# liste des types d'unité
quantity = ["g", "ml", "kg", "cl", "cm", "l", "dl", "batons","grams", "kilograms", "lb", "lbs", "pounds", "c.à.s." ,"c.à.c." "grammes", "cs", "cc", "verre", "branche", "branches", "verres", "cuillères", "poignée", "poignées", "gousse", "gousses", "cuillère", "sachet", "sachets", "rouleau", "rouleaux", "pincée", "pincées", "litre", "litres", "filaments", "filament", "cubes", "cube", "paquet", "paquets", "boîte", "boîtes","boites","boite", "brique", "briques", "pot","pots", "feuilles", "feuille", "morceau", "morceaux", "zestes", "zeste", "boules", "boule", "tête", "têtes", "bol", "bols", "bouquet", "bouquets", "portion", "portions"]


In [76]:
nlp = spacy.load("fr_core_news_sm")

@Language.factory('ruler2')
def Ruler2(nlp, name):

    digit_pattern = [    
        {"POS":"NUM"}
    ]
    
    weights_pattern = [
    {"LIKE_NUM": True},    
    
    {"LOWER": {"IN": quantity}}
    ]
    
    ingredient_pattern = [
        {"LOWER": {"IN": ingredients}}
    ]
    
    verbe_pattern = [
        {"LOWER": {"IN": verbes}}
    ]
        
    ustensile_pattern = [
        {"LOWER": {"IN": ustensiles}}
    ] 
    
   
    patterns = [{"label": "INGREDIENT", "pattern": ingredient_pattern}, {"label": "ACTION", "pattern": verbe_pattern}, {"label": "USTENSILE", "pattern": ustensile_pattern},{"label": "QUANTITY", "pattern": weights_pattern},{"label": "UNIT_QUANTITY", "pattern": digit_pattern}]
  
    return EntityRuler(nlp, patterns=patterns)

nlp.add_pipe('ruler2', before="ner")


In [77]:
len(textNorm)

230494

In [82]:
# On crée nos data 80% train et 20% test: 184360 vs 46134
corpus = textNorm[0:184359]

In [83]:
TRAIN_DATA = []

#iterate over the corpus again and create spacy 2 data input (will be convert later)
for sentence in corpus:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary 
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        #print(ent.text,ent.start_char, ent.end_char,ent.label_)
    if len(entities) > 0:
        TRAIN_DATA.append([sentence, {"entities": entities}])
        


print (TRAIN_DATA[2])

['découp escalop poulet petit morceau enlev mélang prépar', {'entities': [[0, 6, 'ACTION'], [7, 14, 'INGREDIENT'], [15, 21, 'INGREDIENT'], [42, 48, 'ACTION'], [49, 55, 'ACTION']]}]


In [84]:
print (TRAIN_DATA[20])

['vers mélang rempl entier terrin lut étanchéïfi couvercl terrin pât farin eau cuir four moyen thermostat 5 6 1 30 1 45 refroid roug charpent madiran cahor hermitag corn', {'entities': [[5, 11, 'ACTION'], [25, 31, 'INGREDIENT'], [32, 35, 'ACTION'], [47, 55, 'USTENSILE'], [56, 62, 'USTENSILE'], [63, 66, 'INGREDIENT'], [67, 72, 'INGREDIENT'], [73, 76, 'INGREDIENT'], [82, 86, 'USTENSILE'], [104, 105, 'UNIT_QUANTITY'], [163, 167, 'USTENSILE']]}]


In [85]:
print('Après traitement des données, la construction du data set donne: ',len(TRAIN_DATA),' données pour le dataTrain \n\n')

Après traitement des données, la construction du data set donne:  172727  données pour le dataTrain 




In [102]:
# On crée nos data 80% train et 20% test: 184360 vs 46134
corpusTest = textNorm[184360:230493]

In [103]:
TEST_DATA = []

#iterate over the corpus again
for sentence in corpusTest:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary 
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities) > 0:
        TEST_DATA.append([sentence, {"entities": entities}])
        

print (TEST_DATA[0])

['préchauff four 175°c thermostat 5 6', {'entities': [[0, 9, 'ACTION'], [10, 14, 'USTENSILE'], [15, 18, 'UNIT_QUANTITY'], [32, 33, 'UNIT_QUANTITY']]}]


In [104]:
print('Après traitement des données, la construction du data set donne: ',len(TEST_DATA),' données pour le dataTest \n\n')

Après traitement des données, la construction du data set donne:  43095  données pour le dataTest 




In [105]:
# Méthode pour sauvegarde de fichier json
def save_data(file,data):
    with open(file,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4)

In [106]:
listPhrasesTitresNorm[0:10]
listPhrasesTitres[0:10]

['poulet tandoori rouge',
 'lasagnes végétariennes  ( facile ) ',
 'terrine de foie de porc',
 'lapin au vin blanc  ( à la cocotte ) ',
 'escalopes lucullus à la cocotte',
 'tarte aux oignons et carré frais',
 'coq au vin facile',
 'délicieuse pomme de terre du dimanche',
 'pavé de saumon grillé à la confiture de safran',
 'pigeons à la bière']

In [107]:
corpusTitresTest = listPhrasesTitresNorm[0:23225]


In [108]:
TEST_TITRE_DATA = []

#iterate over the corpus again
for sentence in corpusTitresTest:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary 
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities) > 0:
        TEST_TITRE_DATA.append([sentence, {"entities": entities}])
        
   

print (TEST_TITRE_DATA[0])

['poulet tandoor roug', {'entities': [[0, 6, 'INGREDIENT'], [7, 14, 'USTENSILE']]}]


In [109]:
print('Après traitement des données, la construction du data set donne: ',len(TEST_TITRE_DATA),' données pour le data d\'évaluation sur les titres\n\n')


Après traitement des données, la construction du data set donne:  22074  données pour le data d'évaluation sur les titres




## Sauvegarde des données pour entrainement: train/test/eval




In [110]:
save_data("./outputData/ner_training_data.json",TRAIN_DATA)

In [111]:
save_data("./outputData/ner_test_data.json",TEST_DATA)

In [112]:
save_data("./outputData/nerTitre_test_data.json",TEST_TITRE_DATA)

### Dépendance Verbes/Ingrédients

In [ ]:
# Test intéressant mais comportant des erreurs

In [96]:
var =  df['preparation'][0:100].values.tolist().lower()
var = ''.join(var)

In [97]:
doc2 = nlp(var)

In [99]:
sents = [sent for sent in doc2.sents]
#sents

[A préparer la veille !,
 Mélanger l'épice Tandoori (épice indienne rouge),
 , le lait ou de yaourt grec, l'huile et le jus de citron.,
 Découper les escalopes de poulet en petits morceaux (enlever le gras) et les mélanger à la préparation.,
 
     Laisser reposer 24h au frais.,
 
     Faire griller,
 les morceaux à la poêle (ou rôtir en brochettes) et servir avec une sauce à la menthe.,
 Facile et délicieux !,
 Excellent aussi avec d'autres épices (Tikka par exemple).,
 C'est une excellente entrée, ou encore un plat avec une salade, des tranches crues d'oignons doux.,
 Vin rouge d'Australie, NZ ou Afrique du SudSi vous utilisez des oignons, faites,
 -les revenir (dans une sauteuse ou un wok) jusqu'à ce qu'ils soient fondants.,
 
     ,
 Coupez les tomates, ajoutez,
 -les aux oignons, puis faites mijoter  avec des herbes de Provence (n'hésitez pas !,
 ), sel et poivre.,
 
     ,
 Coupez les courgettes en rondelles puis incorporez,
 -les au mélange.,
 
     Ajoutez de la sauce tomate (o

In [100]:
dicIngrVerb = {}
for sent in sents:
    for entity in sent.ents:
        print(entity.text,entity.start_char, entity.end_char,entity.label_)
        if entity.label_ == 'INGREDIENT':
            for chunk in sent.noun_chunks:
                   if chunk.root.dep_ in ['obj','conj']:
                        if chunk.root.head.text != ' ':
                            if entity.text in chunk.text:
                                if (entity.text not in dicIngrVerb):
                                    dicIngrVerb[entity.text] = list()
                                    dicIngrVerb[entity.text].append(chunk.root.head.text) 
                                else:
                                    dicIngrVerb[entity.text].append(chunk.root.head.text)

                                print([chunk.text, chunk.root.text, chunk.root.dep_,chunk.root.head.text])


Mélanger 22 30 PER
Tandoori 39 47 PER
lait 75 79 INGREDIENT
yaourt 86 92 INGREDIENT
grec 93 97 INGREDIENT
jus 113 116 INGREDIENT
citron 120 126 INGREDIENT
Découper 127 135 PER
poulet 153 159 INGREDIENT
['les escalopes de poulet en petits morceaux (enlever le gras)', 'escalopes', 'obj', 'Découper']
gras 191 195 INGREDIENT
['les escalopes de poulet en petits morceaux (enlever le gras)', 'escalopes', 'obj', 'Découper']
frais 258 263 ACTION
Facile 369 375 PER
Tikka 429 434 PER
par 435 438 ACTION
plat 491 495 USTENSILE
Vin 548 551 INGREDIENT
NZ 571 573 LOC
Afrique du SudSi 577 593 LOC
wok 665 668 USTENSILE
Provence 794 802 LOC
sel 822 825 INGREDIENT
Ajoutez 912 919 LOC
1 cuillère 980 990 QUANTITY
20 1096 1098 UNIT_QUANTITY
plat 1193 1197 USTENSILE
1 1200 1201 UNIT_QUANTITY
1 1222 1223 UNIT_QUANTITY
1 1247 1248 UNIT_QUANTITY
béchamel 1259 1267 INGREDIENT
1 1269 1270 UNIT_QUANTITY
sel 1332 1335 INGREDIENT
four 1426 1430 USTENSILE
7 1449 1450 UNIT_QUANTITY
C 1455 1456 MISC
35 1459 1461 UNIT_QU

3/ Laisser 11962 11972 PER
torchon 12022 12029 INGREDIENT
['un torchon humide', 'torchon', 'obj', 'posant']
4/ 12103 12105 UNIT_QUANTITY
Une fois la pâte levée 12106 12128 MISC
trois 12145 12150 UNIT_QUANTITY
Passer 12246 12252 MISC
œuf 12286 12289 INGREDIENT
["un mélange de jaune d'œuf et d'un peu de lait", 'mélange', 'obj', 'Passer']
lait 12305 12309 INGREDIENT
["un mélange de jaune d'œuf et d'un peu de lait", 'mélange', 'obj', 'Passer']
pavot 12377 12382 INGREDIENT
['la tresse de graines de sésame ou de pavot', 'tresse', 'obj', 'saupoudrer']
bleu 12383 12387 INGREDIENT
5/ 12399 12401 UNIT_QUANTITY
Laisser 12402 12409 PER
1/4 12418 12421 UNIT_QUANTITY
four 12448 12452 USTENSILE
190 12455 12458 UNIT_QUANTITY
6 12467 12468 UNIT_QUANTITY
35 12478 12480 UNIT_QUANTITY
Faire 12623 12628 PER
caramel 12704 12711 INGREDIENT
eau 12831 12834 INGREDIENT
nuoc 12855 12859 INGREDIENT
['le nuoc', 'nuoc', 'obj', 'ajouter']
mam 12860 12863 INGREDIENT
Servir 13008 13014 LOC
riz 13023 13026 INGREDIENT
b

2 cuillère 22168 22178 QUANTITY
ail 22224 22227 INGREDIENT
["l'ail et l'oignon", 'ail', 'obj', 'revenir']
oignon 22233 22239 INGREDIENT
["l'ail et l'oignon", 'ail', 'obj', 'revenir']
5 22248 22249 UNIT_QUANTITY
par 22301 22304 ACTION
10 22333 22335 UNIT_QUANTITY
15 22338 22340 UNIT_QUANTITY
Eplucher 22376 22384 PER
poivron 22555 22562 INGREDIENT
Mettre 22581 22587 PER
3 litres 22620 22628 QUANTITY
eau 22631 22634 INGREDIENT
sel 22644 22647 INGREDIENT
curry 22654 22659 INGREDIENT
40min 22675 22680 UNIT_QUANTITY
un gousse 23004 23013 QUANTITY
ail 23016 23019 INGREDIENT
Ajoutez 23074 23081 LOC
saint-Jacques 23108 23121 LOC
Laissez 23140 23147 PER
Ajoutez 23181 23188 LOC
eau 23191 23194 INGREDIENT
["l'eau de vos palourdes à la préparation", 'eau', 'obj', 'Ajoutez']
Ajoutez 23256 23263 LOC
nuoc 23301 23305 INGREDIENT
['une cuillère à soupe de nuoc man', 'cuillère', 'obj', 'Ajoutez']
man 23306 23309 INGREDIENT
['une cuillère à soupe de nuoc man', 'cuillère', 'obj', 'Ajoutez']
piment 23340 23

25 33004 33006 UNIT_QUANTITY
210 33017 33020 UNIT_QUANTITY
C 33021 33022 MISC
7 33035 33036 UNIT_QUANTITY
four 33046 33050 USTENSILE
lait 33134 33138 INGREDIENT
Réalisez 33263 33271 PER
8 33280 33281 UNIT_QUANTITY
2 33296 33297 UNIT_QUANTITY
par 33298 33301 ACTION
Laissez 33317 33324 MISC
4 33453 33454 UNIT_QUANTITY
frais 33464 33469 ACTION
jambon 33482 33488 INGREDIENT
ail 33556 33559 INGREDIENT
["et l'ail qui doit être haché", 'ail', 'conj', 'adaptée']
un bol 33612 33618 QUANTITY
Sortez 33655 33661 LOC
Mettez 33771 33777 LOC
brin 33891 33895 INGREDIENT
coup 33920 33924 ACTION
deux 33953 33957 UNIT_QUANTITY
par 33966 33969 ACTION
Laver 34118 34123 LOC
5 34267 34268 UNIT_QUANTITY
4 34291 34292 UNIT_QUANTITY
film 34306 34310 ACTION
Peler 34463 34468 PER
ail 34481 34484 INGREDIENT
["la gousse d'ail", 'gousse', 'obj', 'Peler']
Hacher 34523 34529 PER
frais 34572 34577 ACTION
CARRES 34587 34593 ORG
FRAIS 34594 34599 ACTION
1 cuillère 34642 34652 QUANTITY
cerfeuil 34715 34723 INGREDIENT
ail 

Mettre 43814 43820 PER
lait 43970 43974 INGREDIENT
pain 44130 44134 INGREDIENT
lard 44185 44189 ACTION
comtois 44241 44248 INGREDIENT
deux 44315 44319 UNIT_QUANTITY
vin 44329 44332 INGREDIENT
Loire 44385 44390 LOC
vin 44424 44427 INGREDIENT
eau 44440 44443 INGREDIENT
1 bouquet 44455 44464 QUANTITY
sel 44475 44478 INGREDIENT
1 44547 44548 UNIT_QUANTITY
oignon 44554 44560 INGREDIENT
écumoir 44621 44628 USTENSILE
par 44642 44645 ACTION
bouillon 44740 44748 INGREDIENT
['le bouillon de poisson', 'bouillon', 'obj', 'Passer']
poisson 44752 44759 INGREDIENT
['le bouillon de poisson', 'bouillon', 'obj', 'Passer']
eau 44840 44843 INGREDIENT
Porter 44863 44869 LOC
Servir 44935 44941 LOC
Farcir 44965 44971 LOC
un verre 45081 45089 QUANTITY
marc 45093 45097 INGREDIENT
["un verre de marc d'Arbois", 'verre', 'obj', 'Ajouter']
Arbois 45100 45106 INGREDIENT
["un verre de marc d'Arbois", 'verre', 'obj', 'Ajouter']
Continuer 45119 45128 PER
Retirer 45145 45152 PER
plat 45174 45178 USTENSILE
jus 45217 452

['le lait', 'lait', 'obj', 'Ajoutez']
basilic 55711 55718 INGREDIENT
["la casserole du feu ajoutez-y les pointes d'asperge, les crevettes et les feuilles de basilic ciselés", 'casserole', 'obj', 'retirer']
8 55767 55768 UNIT_QUANTITY
four 55776 55780 USTENSILE
Parsemez 55859 55867 PER
parmesan 55871 55879 INGREDIENT
four 55913 55917 USTENSILE
10 55933 55935 UNIT_QUANTITY
pinot 56003 56008 INGREDIENT
2 56037 56038 UNIT_QUANTITY
pintadeau 56063 56072 INGREDIENT
['le pintadeau', 'pintadeau', 'obj', 'ajouter']
cognac 56166 56172 INGREDIENT
['le cognac', 'cognac', 'obj', 'Verser']
pintadeau 56199 56208 INGREDIENT
['le pintadeau', 'pintadeau', 'obj', 'arrosant']
bouillon 56333 56341 INGREDIENT
['toutes les plumes restantes).Ajouter les échalotes ou les petits oignons et verser le bouillon et ajouter le thym', 'plumes', 'obj', 'ôter']
thym 56356 56360 INGREDIENT
['toutes les plumes restantes).Ajouter les échalotes ou les petits oignons et verser le bouillon et ajouter le thym', 'plumes', 'obj

veau 64506 64510 INGREDIENT
["tous les éléments : la viande de veau, les lardons, le pied de veau, l'oignon et les 2 carottes, le bouquet garni, sel et poivre", 'éléments', 'obj', 'mettez']
pied 64528 64532 INGREDIENT
["tous les éléments : la viande de veau, les lardons, le pied de veau, l'oignon et les 2 carottes, le bouquet garni, sel et poivre", 'éléments', 'obj', 'mettez']
veau 64536 64540 INGREDIENT
["tous les éléments : la viande de veau, les lardons, le pied de veau, l'oignon et les 2 carottes, le bouquet garni, sel et poivre", 'éléments', 'obj', 'mettez']
oignon 64544 64550 INGREDIENT
["tous les éléments : la viande de veau, les lardons, le pied de veau, l'oignon et les 2 carottes, le bouquet garni, sel et poivre", 'éléments', 'obj', 'mettez']
2 64558 64559 UNIT_QUANTITY
bouquet 64573 64580 INGREDIENT
["tous les éléments : la viande de veau, les lardons, le pied de veau, l'oignon et les 2 carottes, le bouquet garni, sel et poivre", 'éléments', 'obj', 'mettez']
sel 64588 64591 I

In [101]:
dicIngrVerb

{'poulet': ['Découper',
  'Ouvrir',
  'remettre',
  'Couper',
  'couvrir',
  'mariner',
  'mettre'],
 'gras': ['Découper'],
 'lapin': ['Ajouter'],
 'vin': ['Verser', 'Ajouter', 'Ajouter', 'ajouter', 'ajouter', 'Incorporer'],
 'oignon': ['Coupez',
  'Coupez',
  'finement',
  'Mettre',
  'revenir',
  'Emincer',
  'hacher',
  'Pelez',
  'mettez'],
 'jus': ['recouvrez',
  'Ajouter',
  'ajouter',
  'Ajouter',
  'Faire',
  'Ajouter',
  'retirer',
  'ajouter',
  'Ajouter',
  'ajouter',
  '\n    ',
  'Ajouter',
  'Ajouter',
  'faire',
  'Ajouter',
  'rajouter',
  'Ajouter'],
 'sel': ['ajoutez',
  'mettre',
  'ajouter',
  'Faire',
  'Garnir',
  'ensemble',
  'ramolli',
  'Préparez',
  'ajouter',
  'suer',
  'ajouter',
  'mettez'],
 'Cognac': ['Ajouter'],
 'roquefort': ['Ajouter'],
 'safran': ['ajouter'],
 'saumon': ['entendu',
  'entendu',
  'positionner',
  'Déposer',
  'Désolée',
  'Tartiner'],
 'filet': ['cuirePrendre', 'Ajouter', 'verser', 'Ajouter'],
 'mignon': ['cuirePrendre', 'Ajouter'],